In [7]:
from fs.zipfs import ZipFS
import fs.path
import json
import numpy as np

orientation_edits = {}
with open("/scr/ig_pipeline/metadata/orientation_edits.zip", "rb") as f:
    with ZipFS(f) as orientation_zip_fs:
        for item in orientation_zip_fs.glob("recorded_orientation/*/*.json"):
            model = fs.path.splitext(fs.path.basename(item.path))[0]
            orientation = json.loads(orientation_zip_fs.readtext(item.path))[0]
            if np.allclose(orientation, [0, 0, 0, 1], atol=1e-3):
                continue
            orientation_edits[model] = orientation

In [8]:
len(orientation_edits)

65

In [11]:
# Find objects that have new orientation edits
import glob, os
orientation_edits_2 = {}
scale_edits_2 = {}
for item in glob.glob("/scr/ig_pipeline/qa-merged/*/*.json"):
    model = os.path.splitext(os.path.basename(item))[0]
    with open(item) as f:
      data = json.load(f)
      orientation = data["orientation"]
      scale = data["scale"]
    if not np.allclose(orientation, [0, 0, 0, 1], atol=1e-3):
      orientation_edits_2[model] = orientation
    if not np.allclose(scale, [1, 1, 1], atol=1e-3):
      scale_edits_2[model] = scale

In [6]:
# Overlapping keys
set(orientation_edits) & set(orientation_edits_2)

{'aegxpb',
 'azoiaq',
 'bedkqu',
 'cdmmwy',
 'cprjvq',
 'cvdbum',
 'dhnxww',
 'hacehh',
 'icpews',
 'jwxbpa',
 'msfzpz',
 'otyngn',
 'pkfydm',
 'pobfpe',
 'qatgeb',
 'qlxhhh',
 'snvhlz',
 'twknia',
 'wopjex',
 'xjdyon',
 'ykfkyq',
 'ynwamu',
 'ysdoep',
 'ziomqg'}

In [12]:
scale_edits_2

{'gpbmnk': [1.2100000000000002, 1.2100000000000002, 1.2100000000000002],
 'qqnarm': [1.4641000000000006, 1.4641000000000006, 1.4641000000000006],
 'xkvwti': [1.1, 1.1, 1.1],
 'todoph': [1.1, 1.1, 1.1],
 'vkrjps': [1.6105100000000008, 1.6105100000000008, 1.6105100000000008],
 'qbkrzg': [2.357947691000002, 2.357947691000002, 2.357947691000002],
 'qofuoa': [2.1435888100000016, 2.1435888100000016, 2.1435888100000016],
 'omzprq': [1.2100000000000002, 1.2100000000000002, 1.2100000000000002],
 'hwrflj': [1.4641000000000006, 1.4641000000000006, 1.4641000000000006],
 'bwteqh': [1.3310000000000006, 1.3310000000000006, 1.3310000000000006],
 'zutnsf': [1.6105100000000008, 1.6105100000000008, 1.6105100000000008],
 'rizrsp': [2.357947691000002, 2.357947691000002, 2.357947691000002],
 'netbsb': [1.3310000000000004, 1.3310000000000004, 1.3310000000000004],
 'yyuiva': [1.2100000000000002, 1.2100000000000002, 1.2100000000000002],
 'zlxfxt': [1.771561000000001, 1.771561000000001, 1.771561000000001],
 'ob

In [13]:
# How many scale edits are there?
len(scale_edits_2)

563

In [14]:
# How many of them are scene objects?
with open("/scr/ig_pipeline/artifacts/pipeline/object_inventory.json") as f:
  providers = {k.split("-")[1]: v for k, v in json.load(f)["providers"].items()}

scene_scale_edits_2 = [x for x in scale_edits_2 if providers[x].startswith("scenes")]

In [15]:
len(scene_scale_edits_2)

249

In [17]:
# Whats the mean magnitude of the general vs the scene?
print("General", np.median([np.linalg.norm(x) for x in scale_edits_2.values()]))
print("Scene", np.median([np.linalg.norm(scale_edits_2[x]) for x in scene_scale_edits_2]))

General 2.3053596248741766
Scene 1.9052558883257653


In [21]:
# Count edits by target
import collections
scale_edits_2_counter = collections.Counter([providers[x] for x in scale_edits_2])
scale_edits_2_counter.most_common()

[('scenes/house_single_floor', 91),
 ('objects/legacy_batch-05', 50),
 ('objects/legacy_batch-03', 31),
 ('objects/legacy_batch-04', 25),
 ('objects/legacy_batch-10', 21),
 ('objects/legacy_batch-09', 20),
 ('objects/legacy_batch-08', 20),
 ('objects/legacy_batch-06', 19),
 ('objects/legacy_batch-07', 19),
 ('scenes/grocery_store_half_stocked', 16),
 ('objects/legacy_batch-11', 16),
 ('scenes/house_double_floor_upper', 15),
 ('scenes/grocery_store_asian', 12),
 ('scenes/grocery_store_convenience', 12),
 ('scenes/grocery_store_cafe', 11),
 ('objects/batch-02', 11),
 ('scenes/restaurant_asian', 10),
 ('objects/batch-01', 9),
 ('objects/batch-09', 7),
 ('objects/batch-10', 7),
 ('objects/batch-03', 7),
 ('scenes/hotel_gym_spa', 7),
 ('objects/legacy_batch-00', 6),
 ('scenes/Wainscott_0_garden', 6),
 ('objects/batch-06', 6),
 ('scenes/restaurant_brunch', 6),
 ('objects/legacy_batch-01', 6),
 ('scenes/house_double_floor_lower', 6),
 ('objects/batch-12', 6),
 ('scenes/office_large', 5),
 ('s